In [ ]:
%pip install -q -U transformers
%pip install -q bitsandbytes accelerate>=0.25.0
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import fsspec
from PIL import Image
from io import BytesIO


# Общее описание экспериментов
## Определить картину по жанру
Для экономии данных, скачан не весь датасет, а лишь ~1000 записей.
Дополнительно был скачан файл с описанием лейблов (для артистов, жанров, стилей). Они были преобразованы в словари {label: name}

In [ ]:
import pandas as pd
import fsspec

file_url = 'https://huggingface.co/datasets/huggan/wikiart/resolve/main/data/train-00000-of-00072.parquet'
fs = fsspec.filesystem('https')

# Читаем данные прямо с URL, не загружая на локальный диск
with fs.open(file_url) as f:
    df = pd.read_parquet(f)

Пример картины

In [ ]:
from PIL import Image
from io import BytesIO

# Предположим, что limited_ds[0]['image']['bytes'] содержит байты изображения
image_bytes = df.loc[0]['image']['bytes']

# Преобразуем байты в поток
image_stream = BytesIO(image_bytes)

# Используем PIL для открытия изображения
image = Image.open(image_stream)

# Отображение изображения
# image


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch

model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Проверка на одной из картин

In [ ]:

# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image")
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "What is on the picture"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor(images=image, text=prompt, return_tensors='pt').to(device, torch.float16)

output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
print(processor.decode(output[0], skip_special_tokens=True))


USER:  
What is on the picture ASSISTANT: The picture features a beautiful landscape with a body of water, such as a lake or a river, surrounded by trees. There are several birds flying in the sky, adding to the serene atmosphere of the scene. The image is a painting, which captures the essence of nature and the tranquility of the environment.


In [ ]:
!nvidia-smi

Wed Oct  9 12:48:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0              27W /  70W |  13593MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import requests
import json

# URL файла JSON с метаданными
file_url = 'https://huggingface.co/datasets/huggan/wikiart/resolve/main/dataset_infos.json'

# Загрузка файла JSON с помощью requests
response = requests.get(file_url)

# Преобразуем содержимое в формат JSON
dataset_info = json.loads(response.content)

# Выводим содержимое JSON для анализа
# print(json.dumps(dataset_info, indent=4))


In [ ]:
import random

genres_info = dataset_info['huggan--wikiart']['features']['genre']
genres = genres_info['names']
artists =  dataset_info['huggan--wikiart']['features']['artist']['names']
artists_dict = {i: author for i, author in enumerate(artists)}
styles =  dataset_info['huggan--wikiart']['features']['style']['names']
styles_dict = {i: s for i, s in enumerate(styles)}

print(artists_dict)
print(styles_dict)
genres = list([f'{i}: {g}' for i, g in enumerate(genres)])

{0: 'Unknown Artist', 1: 'boris-kustodiev', 2: 'camille-pissarro', 3: 'childe-hassam', 4: 'claude-monet', 5: 'edgar-degas', 6: 'eugene-boudin', 7: 'gustave-dore', 8: 'ilya-repin', 9: 'ivan-aivazovsky', 10: 'ivan-shishkin', 11: 'john-singer-sargent', 12: 'marc-chagall', 13: 'martiros-saryan', 14: 'nicholas-roerich', 15: 'pablo-picasso', 16: 'paul-cezanne', 17: 'pierre-auguste-renoir', 18: 'pyotr-konchalovsky', 19: 'raphael-kirchner', 20: 'rembrandt', 21: 'salvador-dali', 22: 'vincent-van-gogh', 23: 'hieronymus-bosch', 24: 'leonardo-da-vinci', 25: 'albrecht-durer', 26: 'edouard-cortes', 27: 'sam-francis', 28: 'juan-gris', 29: 'lucas-cranach-the-elder', 30: 'paul-gauguin', 31: 'konstantin-makovsky', 32: 'egon-schiele', 33: 'thomas-eakins', 34: 'gustave-moreau', 35: 'francisco-goya', 36: 'edvard-munch', 37: 'henri-matisse', 38: 'fra-angelico', 39: 'maxime-maufra', 40: 'jan-matejko', 41: 'mstislav-dobuzhinsky', 42: 'alfred-sisley', 43: 'mary-cassatt', 44: 'gustave-loiseau', 45: 'fernando-bo

In [ ]:
prompt_template = """USER: <image>\nThere are 11 possible genres:\n{genres_str}.\nWhat is the genre of painting by {artist} in style {style}? Choose from the given list. Say only the number of genre, do not output anything else.\nASSISTANT:"""

In [ ]:
prompt_t = """The task is to classify an image into one of the following 11 genres:\n{genres_str}.\nYou are provided with the name of the artist: and the painting style of the image.\nArtist: {artist}\nStyle: {style}\nUse all your knowledge and given information to determine the correct genre. Choose the genre from the list by providing only the corresponding number. Say ONLY the number of the genre and do not say anything else."""

In [ ]:
prompt_t = """There are 11 possible genres:\n{genres_str}.\nWhat is the genre of painting by {artist} in style {style}? Write an analysis of the genre the image, use all your knowledge. Then choose the genre from the given list. In the end say the number of genre."""

In [ ]:
import time


times=[]
# token=[]
# tok_per_second=[]
predictions = []
correct_predictions = 0
total_predictions = len(df)
true_genres = df['genre'].tolist()

for idx, row in df.loc[:5].iterrows():
    print(idx)

    image_bytes = row['image']['bytes']
    image_stream = BytesIO(image_bytes)
    image = Image.open(image_stream)
    # print(type(image))
    artist = artists_dict[row['artist']]
    style = styles_dict[row['style']]
    shuffle(genres)
    genres_str = '\n'.join(genres)
    prompt = prompt_t.format(genres_str=genres_str, artist=artist, style=style)

    start_time = time.time()
    messages = [
      { "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image"},
          ],
      },
    ]
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    # print(input_text)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    # print(inputs.keys())
    # print(inputs['pixel_values'].shape)

    outputs = model.generate(**inputs, max_new_tokens=100)
    del inputs
    # End generation
    end_time = time.time() - start_time
    print(f'Time per message: {end_time:.2f} s')
    times.append(end_time)
    outputs_text = processor.decode(outputs[0])
    print(outputs_text)
    del outputs
    regexp_res = re.search(r'(\d+)</s>', outputs_text)
    if regexp_res:
        answer = regexp_res.group(1)
    else:
        answer = '12'

    pred = int(eval(answer))

    # count_token = len(tokenizer(outputs_text, return_tensors="pt")['input_ids'][0].to("cpu"))
    # print(f'Token per message: {count_token}')
    # token.append(count_token)

    # token_per_second = count_token / end_time
    # print(f'Token per second: {token_per_second}')
    # tok_per_second.append(token_per_second)

    true_genre = row['genre']
    print(pred, true_genre)
    if pred == true_genre:
        correct_predictions += 1

    predictions.append(pred)

0
Time per message: 8.04 s
USER: <image> 
There are 11 possible genres:
7: religious_painting
0: abstract_painting
3: illustration
6: portrait
4: landscape
1: cityscape
8: sketch_and_study
9: still_life
10: Unknown Genre
2: genre_painting
5: nude_painting.
What is the genre of painting by vincent-van-gogh in style Realism? Write an analysis of the genre the image, use all your knowledge. Then choose the genre from the given list. In the end say the number of genre. ASSISTANT: The painting by Vincent Van Gogh is a realistic depiction of a landscape scene, featuring a body of water, trees, and a bird flying overhead. The artist's use of light and shadow, as well as the attention to detail in the representation of the natural elements, contribute to the realistic style of the painting. The genre of this painting can be classified as landscape genre painting.</s>
12 4
1


In [ ]:
len(predictions)

1132

In [ ]:
correct_predictions

281

In [ ]:
df_results = pd.DataFrame({
    'prediction': predictions,
    'true': true_genres
})

In [ ]:
len(df_results[df_results.prediction == df_results.true]) / len(df_results)

0.24823321554770317

In [ ]:
for p, t in zip(predictions, true_genres):
  if p != t:
    print(f"Incorrect Pred: {p}, True: {t}")
  else:
    print(f"Pred: {p}, True: {t}")

Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 7
Incorrect Pred: 12, True: 6
Incorrect Pred: 12, True: 2
Incorrect Pred: 12, True: 10
Incorrect Pred: 12, True: 2
Incorrect Pred: 12, True: 10
Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 7
Incorrect Pred: 12, True: 6
Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 2
Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 6
Incorrect Pred: 12, True: 6
Incorrect Pred: 12, True: 9
Incorrect Pred: 12, True: 8
Incorrect Pred: 12, True: 3
Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 3
Incorrect Pred: 12, True: 8
Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 1
Incorrect Pred: 12, True: 8
Incorrect Pred: 12, True: 6
Incorrect Pred: 12, True: 1
Incorrect Pred: 12, True: 4
Incorrect Pred: 12, True: 10
Incorrect Pred: 12, True: 8
Incorrect Pred: 12, True: 5
Incorrect Pred: 12, True: 6
Incorrect Pred: 12, True: 7
Incorrect Pred: 12, True: 9
Incorrect Pred: 12, True: 10
Incorrect Pred: 12, True: 6
Incorrect Pred: 